In [ ]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.mixture import GaussianMixture
from varclushi import VarClusHi
from sklearn.preprocessing import StandardScaler
import sklearn.metrics as metrics
from sklearn.decomposition import KernelPCA
from sklearn.preprocessing import RobustScaler, MinMaxScaler, Normalizer

# 1
В файле «baseball.csv» находится выборка с информацией по игрокам в бейсбол, включая
статистику их результативности, время участия в играх, лига, зарплата и т.д. Name (имя) нужно
считать идентификатором записи. Загрузите этот файл и произведите следующие действия для
кластерного анализа.

In [ ]:
df = pd.read_csv("baseball.csv")
df = df.set_index("Name")

df_base = df.copy()

df.head()

# 2
Обработка пропусков. Переменная Salary (и log Salary) может содержать пропуски,
произведите подстановку пропусков методом согласно вашему варианту. Пересчитайте
logSalary как log(1+Salary), чтобы получить более симметричное распределение.

KnnImputer (neighbors=5)

In [ ]:
knn_imp = KNNImputer(n_neighbors=5)

df[["Salary", "logSalary"]] = knn_imp.fit_transform(df[["Salary", "logSalary"]])

df["logSalary"] = np.log(1 + df["Salary"])

df_after_2 = df.copy()

df.head()

# 3
Нормализация переменных – приведите числовые переменные к близким шкалам с помощью
методов для вашего варианта и закодируйте категориальные с помощью OneHotEncoder.

MaxAbsScaler 

In [ ]:
interval = [
    "CrAtBat", "CrBB", "CrHits", "CrHome", "CrRbi", "CrRuns", "logSalary", "nAssts", "nAtBat", "nBB", "nError",
    "nHits", "nHome", "nOuts", "nRBI", "nRuns", "Salary", "YrMajor"
]

nominal = [
    "Div", "Division", "League", "Position", "Team",
]

df[interval] = MaxAbsScaler().fit_transform(df[interval])

# df = pd.get_dummies(df, columns=nominal)

ohe = OneHotEncoder(sparse_output=False)
ohe.fit(df[nominal])

temp_df = pd.DataFrame(data=ohe.transform(df[nominal]), columns=ohe.get_feature_names_out(), index = df.index)
df.drop(columns=nominal, inplace=True)
df = pd.concat([df, temp_df], axis=1)

df.head()

# 4
С помощью восходящей иерархической кластеризации с выбранными параметрами
расстояния согласно вашему варианту постройте кластерную модель данных и дендрограмму
для топ 20 кластеров.

link=average, dist=euclidean

In [ ]:
def plot_dendrogram(model, **kwargs):
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count
    linkage_matrix = np.column_stack([model.children_, model.distances_, counts]).astype(float)
    dendrogram(linkage_matrix, **kwargs)
    plt.xlabel("Количество наблюдений в узле")


model = AgglomerativeClustering(linkage="average", metric="euclidean", n_clusters=20, compute_distances=True)
model = model.fit(df)
plot_dendrogram(model, truncate_mode="level", p=4)

# 5
Рассчитайте значение критерия pseudoF для вариантов кластеризации 2-20 кластеров,
постройте график зависимости критерия от числа кластеров и выберите оптимальное (первый
локальный пик критерия при обходе от малого числа кластеров к большому). Отметьте точку
на графике. Сколько кластеров получилось?

In [ ]:
def sum_dist_to_center(X):
    center = np.mean(X, axis = 0)
    return ((X - center)**2).values.sum()


def choose_num_clusters(X, max_clust = 30):
    N = X.shape[0]
    Q = sum_dist_to_center(X)
    pseudo_f = np.array([])
    for G in range(2, max_clust + 1):
        clustering = KMeans(n_clusters = G, n_init="auto").fit(X)
        W = 0
        for l in range(G):
            elems = X[clustering.labels_ == l]
            W += sum_dist_to_center(elems)
        fisher_stat = ((Q - W)/(G - 1))/(W/(N - G))
        pseudo_f = np.append(pseudo_f, fisher_stat)
        
    plt.plot(range(2, max_clust + 1), pseudo_f)
    ind_best_clust = np.argmax(pseudo_f)
    plt.scatter(ind_best_clust + 2, pseudo_f[ind_best_clust], color="r", marker="D", s=50)
    plt.xlabel("Number of clusters")
    plt.ylabel("Pseudo-F")
    return ind_best_clust + 2


k = choose_num_clusters(df, max_clust=20)
k

# 6
С помощью метода проекции для вашего варианта постройте отображение на плоскость,
цветом точки укажите номер кластера.

PCA

In [ ]:
pca = PCA(n_components=2)
features = pca.fit_transform(df)

plt.scatter(features[:, 0], features[:, 1], c=model.labels_)
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.show()

# 7
Выполните кластеризацию сферическими кластерами с прототипом методом из вашего
варианта, также постройте проекцию как на шаге 6, определите наиболее типичного
представителя (по имени) в каждом из кластеров.

EM

In [ ]:
gm = GaussianMixture(n_components=k, covariance_type="spherical")

clusters = gm.fit_predict(df)

plt.scatter(features[:, 0], features[:, 1], c=clusters)
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.show()

In [ ]:
df7 = df.copy()
df7["cluster"] = clusters

# for i in range(k):
#     cur_cluster = df7[df7["cluster"] == i]
#     center = np.mean(cur_cluster, axis = 0)
#     print(np.sqrt(((cur_cluster - center) ** 2).sum(axis=1)).sort_values().index[0])

for i in range(k):
    cur_cluster = df7[df7["cluster"] == i]
    center = pd.Series(np.append(gm.means_[i], 0), index=df7.columns)
    print(np.sqrt(((cur_cluster - center) ** 2).sum(axis=1)).sort_values().index[0])

# 8
Реализуйте шаги 3-7 в виде функции или класса. 

In [ ]:
def func8(df8, interval, nominal):
    df = df8.copy()
    
    # 3
    print("task 3")
    df[interval] = MaxAbsScaler().fit_transform(df[interval])

    # df = pd.get_dummies(df, columns=nominal)

    ohe = OneHotEncoder(sparse_output=False)
    ohe.fit(df[nominal])

    temp_df = pd.DataFrame(data=ohe.transform(df[nominal]), columns=ohe.get_feature_names_out(), index = df.index)
    df.drop(columns=nominal, inplace=True)
    df = pd.concat([df, temp_df], axis=1)
    
    # 4
    print("task 4")
    model = AgglomerativeClustering(linkage="average", metric="euclidean", n_clusters=20, compute_distances=True)
    model = model.fit(df)
    plot_dendrogram(model, truncate_mode="level", p=4)
    plt.show()
    
    # 5
    print("task 5")
    k = choose_num_clusters(df, max_clust=20)
    plt.show()
    print(k)
    
    # 6
    print("task 6")
    pca = PCA(n_components=2)
    features = pca.fit_transform(df)

    plt.scatter(features[:, 0], features[:, 1], c=model.labels_)
    plt.xlabel("PC1")
    plt.ylabel("PC2")
    plt.show()
    
    # 7
    print("task 7")
    gm = GaussianMixture(n_components=k, covariance_type="spherical")

    clusters = gm.fit_predict(df)

    plt.scatter(features[:, 0], features[:, 1], c=clusters)
    plt.xlabel("PC1")
    plt.ylabel("PC2")
    plt.show()
    
    df7 = df.copy()
    df7["cluster"] = clusters

    # for i in range(k):
    #     cur_cluster = df7[df7["cluster"] == i]
    #     center = np.mean(cur_cluster, axis = 0)
    #     print(np.sqrt(((cur_cluster - center) ** 2).sum(axis=1)).sort_values().index[0])
    
    for i in range(k):
        cur_cluster = df7[df7["cluster"] == i]
        center = pd.Series(np.append(gm.means_[i], 0), index=df7.columns)
        print(np.sqrt(((cur_cluster - center) ** 2).sum(axis=1)).sort_values().index[0])

In [ ]:
func8(df_after_2, interval, nominal)

# 9
Произведите дополнительную предобработку набора данных, сделав распределения
переменных более симметричными. Для этого с помощью гисторамм или метода describe в
dataframe или метода skew найдите переменные с одной модой и тяжелым правым хвостом,
примените к ним преобразование log(1+x). Запустите функцию из шага 8. Как изменилось
число кластеров, проекции и лучшие представители. Как считаете, субъективное качество
кластеризации изменилось? Как и почему?

In [ ]:
df10 = df_after_2.copy()

right_skewed_vars = []
for col in interval:
    if df_after_2[col].skew() > 1 and df_after_2[col].mode().count() == 1:
        right_skewed_vars.append(col)
        
for col in right_skewed_vars:
    df_after_2[col] = np.log1p(df_after_2[col])

func8(df_after_2, interval, nominal)

Число кластеров осталось прежним. Проекции примерно остались прежними. Можно предположить, что субъективное качество кластеризации не изменилось значительно. Однако, изменение лучших представителей может указывать на изменения внутрикластерной структуры или на изменения в распределении данных, которые могут быть интересны для дальнейшего анализа.

# 10
Отберите 5 наиболее значимых переменных с помощью метода из вашего варианта. Запустите
функцию из шага 8. Как изменилось число кластеров, проекции и лучшие представители. Как
считаете, субъективное качество кластеризации изменилось? Как и почему?

VarClus

In [ ]:
X = df10[interval]

In [ ]:
nvar = 5

clusters = VarClusHi(X, maxeigval2=0.8, maxclus=nvar)
clusters.varclus()
clusters.info

max_RS_Ratio = clusters.rsquare.sort_values(by=["Cluster", "RS_Ratio"], ascending=[True, False])

vars = []

for i in range(nvar):
    vars.append(max_RS_Ratio[max_RS_Ratio["Cluster"] == i]["Variable"].values[0])
    #print(max_RS_Ratio[max_RS_Ratio["Cluster"] == i]["Variable"].values[0])

print(*vars)

func8(df10[vars + nominal], vars, nominal)

Число кластеров осталось прежним. Проекции примерно остались прежними. Можно предположить, что субъективное качество кластеризации не изменилось значительно. Однако, изменение лучших представителей может указывать на изменения внутрикластерной структуры или на изменения в распределении данных, которые могут быть интересны для дальнейшего анализа.

# 11
«Творческое задание» на поиск аномалий. Загрузите файл mnist_small.csv. Данный набор
данных содержит подмножество эталонного набора данных рукописных цифр MNIST. 5923
картинок 28x28 пикселей с изображением нуля и 76 картинок с изображением шестерки.
Задача состоит в том, чтобы с использованием методов обучения без учителя для своего
варианта построить одноклассовую модель на основе поиска аномалий, которая максимально
хорошо отфильтрует шестерки (как аномалии) от нулей (как основной выборки). Признаки
картинок описываются их координатами (в названии переменных, например «10x12») и
значением яркости точки по этим координатам. Подбирая параметры метода и преобразуя
признаки как посчитаете нужным, но не используя при этом информацию о label, постройте
модель выявления аномалий с ERR меньше 0.2.

KPCA

In [ ]:
def plot_roc(df):
    fpr, tpr, threshold = metrics.roc_curve(df["label"], df["pred"])
    
    plt.plot(fpr, tpr)
    plt.plot([0, 1], [1, 0])
    plt.axvline(x = 0.2, color="red")
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.show()

In [ ]:
df = pd.read_csv("mnist_small.csv")

df['label'] = df['label'].replace(6,1)

X = df.copy()
X = X.set_index("label")

X = MaxAbsScaler().fit_transform(X)
X = pd.DataFrame(X)

kernel_pca = KernelPCA(
    n_components=2,
    kernel="rbf",
    fit_inverse_transform=True,
    n_jobs=-1)

kernel_pca.fit(X)

X_pca = kernel_pca.transform(X)

E = abs(X.values-kernel_pca.inverse_transform(X_pca))

ET = np.apply_along_axis(np.linalg.norm, 1, E)

df["pred"] = ET

plot_roc(df)

# 12
Постройте ROC кривую с ERR. Выведите 4 картинки с числами (28 на 28 пикселей):
- самый типичный “0” – true negative с минимальной аномальностью
- самая аномальная “6” – true positive с максимальной аномальностью
- самый нетипичный “0” – false positive с максимальной аномальностью
- самая неаномальная “6” – false negative с минимальной аномальностью

In [ ]:
plot_roc(df)

In [ ]:
zeros = df[df["label"] == 0]
sixes = df[df["label"] == 1]

plt.imshow(zeros[zeros["pred"] == zeros["pred"].min()].values[0][1:-1].reshape(28, 28))
plt.show()

plt.imshow(zeros[zeros["pred"] == zeros["pred"].max()].values[0][1:-1].reshape(28, 28))
plt.show()

plt.imshow(sixes[sixes["pred"] == sixes["pred"].min()].values[0][1:-1].reshape(28, 28))
plt.show()

plt.imshow(sixes[sixes["pred"] == sixes["pred"].max()].values[0][1:-1].reshape(28, 28))
plt.show()